# Python Website Reader

Mit diesem Notebook werden die folgenden Websites besucht und die News in Markdown angezeigt und können auch als Markdown-file abgespeichert werden.

Der Pfad und der Filename kann in der aufrufenden Function savefile('D:\\temp', 'srf.md', stream) angepasst werden. Aktuell ist die Function auskommentiert.

<mark>**Bitte zuerst die class colors ausführen, damit die Farben für den Output geladen sind.**</mark>

In [8]:
class colors: 
    '''Colors class:reset all colors with colors.reset'''

    reset         = '\033[0m'
    bold          = '\033[01m'
    disable       = '\033[02m'
    underline     = '\033[04m'
    reverse       = '\033[07m'
    strikethrough = '\033[09m'
    invisible     = '\033[08m'

    class fg: 
        ''' Define the foreground-colors
        use as colors.fg.colorname
        '''
        black      = '\033[30m'
        red        = '\033[31m'
        green      = '\033[32m'
        orange     = '\033[33m'
        blue       = '\033[34m'
        purple     = '\033[35m'
        cyan       = '\033[36m'
        lightgrey  = '\033[37m'
        darkgrey   = '\033[90m'
        lightred   = '\033[91m'
        lightgreen = '\033[92m'
        yellow     = '\033[93m'
        lightblue  = '\033[94m'
        pink       = '\033[95m'
        lightcyan  = '\033[96m'

    class bg: 
        ''' Define the background-colors
        use as colors.bg.colorname
        '''
        black     = '\033[40m'
        red       = '\033[41m'
        green     = '\033[42m'
        orange    = '\033[43m'
        blue      = '\033[44m'
        purple    = '\033[45m'
        cyan      = '\033[46m'
        lightgrey = '\033[47m'


## SRF News
Code mit <mark> Run Cell </mark> ausführen um die SRF News anzuzeigen.

In [ ]:
news = ['https://www.srf.ch/news/neuste-beitraege']

def newsreader(url, links=True, images=True, emphasis=True):
    '''Reads the given url and print a markdown'''
    import requests, html2text, re
    from datetime import datetime
    now = datetime.now()

    response = requests.get(url)
    if(response.status_code == 200):

        print(colors.bold + colors.fg.blue +'{1}{0}{4} {2}{0}{3}{0}'.format('\n', '>' * 100, f'NEWS FROM: {site}', '<' * 100, now.strftime("%Y-%m-%d %H:%M:%S")) + colors.reset)

        html = response.content.decode('utf-8')

        md = html2text.HTML2Text()
        md.ignore_links    = not links
        md.ignore_images   = not images
        md.ignore_emphasis = not emphasis
        md.body_width      = 100
        
        data  = md.handle(html)

        #read from '## Inhalt'
        start = data.find('###  Neuste Beiträge')
        end   = data.find('## Footer')
        stream = [(data[start:end])]

        for i in stream:
            if(i is not None):
                line = i.replace('Mit Video\n\n','').replace('Mit Audio\n\n','')
                print('{0}{1}{2}'.format(colors.fg.green, line, colors.reset))

        return stream

def savefile(filepath, filename, stream):
    '''Save the stream to the file'''
    import os

    if not os.path.exists(filepath):
        os.makedirs(filepath)

    file = open(filepath + '\\' + filename, 'w')
    for i in stream:
        file.write(i)

    file.close()

for site in news:
    stream = newsreader(site, False, False)
    #savefile('D:\\temp', 'srf.md', stream)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
2021-03-05 09:36:18 NEWS FROM: https://www.srf.ch/news/neuste-beitraege
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

###  Neuste Beiträge

  * Volkskongress in Peking

China steckt sich hohe Wirtschaftsziele

An der parlamentarischen Plenarversammlung wurde das Konjunkturziel 2021 bekannt: Mehr als sechs
Prozent Wachstum.

  * Digitalisierter Kundendienst

Hallo, ich bin eine Maschine! Wie kann ich Ihnen helfen?

Unternehmen setzen grosse Hoffnungen in sogenannte Chatbots. Diese sollen vermehrt Anfragen von
Kunden beantworten.

  * Gegen Abschaffung

Konsumentenschutz kämpft für Einzahlungs-Postomaten

Auf Ende Juli schafft Postfinance die Postomaten mit Einzahlfunktion ab. Der Konsumentenschutz
leistet Widerstand.

  * UBS-Geschäftsbericht

So viel verdienten die UBS-Chefs im Jahr 2020

Der Jahresgewinn beträgt 6.557 Milliarden F

## Südostschweiz
Code  mit <mark> Run Cell </mark> ausführen um die Südostschweiz News anzuzeigen.

In [ ]:
news = ['https://www.suedostschweiz.ch']

def newsreader(url, links=True, images=True, emphasis=True):
    '''Reads the given url and print a markdown'''
    import requests, html2text
    from datetime import datetime
    now = datetime.now()

    response = requests.get(url)
    if(response.status_code == 200):
        
        print(colors.bold + colors.fg.blue +'{1}{0}{4} {2}{0}{3}{0}'.format('\n', '>' * 100, f'NEWS FROM: {site}', '<' * 100, now.strftime("%Y-%m-%d %H:%M:%S")) + colors.reset)

        html = response.content.decode('utf-8')

        md = html2text.HTML2Text()
        md.ignore_links    = not links
        md.ignore_images   = not images
        md.ignore_emphasis = not emphasis
        md.body_width      = 100
        
        data  = md.handle(html)

        #read from '# Startseite'
        start  = data.find('# Startseite')
        end    = data.find('## 2\. GR Template Menu')
        stream = [(data[start:end])]
        
        for i in stream:
            if(i is not None):
                line = i.replace('ABO\n\n','').replace('Sponsored Content\n\n','').replace('video\n\n','').replace('bilder\n\n','').replace('Mehr\n\n','')
                print('{0}{1}{2}'.format(colors.fg.green, line, colors.reset))

        return stream

def savefile(filepath, filename, stream):
    '''Save the stream to the file'''
    import os

    if not os.path.exists(filepath):
        os.makedirs(filepath)

    file = open(filepath + '\\' + filename, 'w')
    for i in stream:
        file.write(i)

    file.close()

for site in news:
    stream = newsreader(site, False, False)
    #savefile('D:\\temp', 'somedia.md', stream)
